In [12]:
import numpy as np
import pandas as pd
import random
import os
import time
import pickle
from pathlib import Path

from os.path import join, isfile
from os import path, scandir, listdir

from sklearn.preprocessing import StandardScaler, QuantileTransformer
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.metrics import roc_auc_score

import lightgbm as lgb
#import xgboost as xgb
#import catboost as ctb

import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.simplefilter('ignore')

# Parameters

In [13]:
target = 'target'

DEBUG = False

if DEBUG:
    N_ESTIMATORS = 1
    N_SPLITS = 2
    SEED = 2017
    CVSEED = 2017
    EARLY_STOPPING_ROUNDS = 1
    VERBOSE = 100
    #N_ITERS = 2
else:
    N_SPLITS = 5
    N_ESTIMATORS = 20000
    EARLY_STOPPING_ROUNDS = 300
    VERBOSE = 1000
    SEED = 2017
    CVSEED = 2017
    #N_ITERS = 10

In [14]:
def set_seed(seed=2017):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    
set_seed(SEED)

# Datasets

In [15]:
INPUT = Path("../input/tabular-playground-series-oct-2021")

train = pd.DataFrame(pd.read_csv(INPUT / "train.csv")[target])
#test = pd.read_csv(INPUT / "test.csv")
test = pd.DataFrame()
submission = pd.read_csv(INPUT / "sample_submission.csv")

In [16]:
oof = np.load("../input/tps-oct-2021-stacking-lgb-lv2/lgb_oof.npy")**2 +\
         np.load("../input/tps-oct-2021-stacking-logistic-lv2/logistic_oof.npy")**2

pred = np.load("../input/tps-oct-2021-stacking-lgb-lv2/lgb_pred.npy")**2 +\
         np.load("../input/tps-oct-2021-stacking-logistic-lv2/logistic_pred.npy")**2


In [17]:
oof /= 2
pred /= 2

In [18]:
auci = roc_auc_score(train[target], oof)
auci

0.8574027799182755

In [19]:
submission[target] = pred
submission.to_csv("submission.csv", index=False)

submission

,id,target
0,1000000,0.521098
1,1000001,0.065400
2,1000002,0.823300
3,1000003,0.685686
4,1000004,0.080284
...,...,...
499995,1499995,0.896004
499996,1499996,0.673629
499997,1499997,0.090523
499998,1499998,0.259588


# Log
